# Fonctions Utiles 

## Variables Paramètres

In [1]:
KEY = ""
# Get your key on https://developer.riotgames.com/

In [2]:
import requests
import pandas as pd
import json
import time

## Mémoïzation ?
Affin d'éviter de demander plusieurs fois la même chose, on pourrait avoir une classe 'Joueur' qui permet de centraliser les infos, pour que toutes les méthodes y aient accès. Dans ce cas, les fonctions de calcul de données deviendraient des méthodes de cette classe.

In [19]:
class Player:
    def __init__(self, summonerName, accountId):
        self.summonerName = summonerName
        self.accountId = accountId
        self.champions = []
        self.matches = []
    
    def __str__(self):
        return self.summonerName
    
    def __repr__(self):
        return self.summonerName

## Gestion des "bad requests"

In [4]:
def badRequestsHandler(url):
    """
    Permet d'analyser le code de retour de la requête pour gérer les problèmes éventuels.
    Pas sûr que cela permette de gérer tous les problèmes, certains seront probablement spécifiques.
    """
    r = requests.get(url)
    while r.status_code == 429: #éventuellement à revoir
        time.sleep(5)
        r = requests.get(url)
    if r.status_code == 400:
        raise Exception("Requête invalide")
    elif r.status_code == 401:
        raise Exception("Unauthorized")
    elif r.status_code == 403:
        raise Exception("Non autorisé: vérifiez la clé")
    elif r.status_code == 404:
        raise Exception("Données non trouvées")
    elif r.status_code == 405:
        raise Exception("Méthode non autorisée")
    elif r.status_code == 415:
        raise Exception("Unsupported media type")
    elif r.status_code == 500:
        raise Exception("Internal server error")
    elif r.status_code == 502:
        raise Exception("Bad gateway")
    elif r.status_code == 503:
        raise Exception("Service non disponible")
    elif r.status_code == 504:
        raise Exception("Gateway timeout")
    elif r.status_code == 200:
        return r.json()

## Récupération des données brutes d'un joueur

In [5]:
def requestSummonerInfo(summoner_name,key):
    """
    Renvoie un dictionnaire contenant des infos sur le joueur 'summoner_name', à savoir :
    accountId :    string - Encrypted account ID. Max length 56 characters.
    profileIconId:    int - ID of the summoner icon associated with the summoner.
    revisionDate:    long - Date summoner was last modified specified as epoch milliseconds. The following events will update this timestamp: summoner name change, summoner level change, or profile icon change.
    name:          string - Summoner name.
    id:            string - Encrypted summoner ID. Max length 63 characters.
    puuid:         string - Encrypted PUUID. Exact length of 78 characters.
    summonerLevel:   long - Summoner level associated with the summoner.
    """
    return badRequestsHandler(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={key}")

requestSummonerInfo('Skeanz',KEY)

{'id': 'MsJ0SiJEwktncbZVgBlW1uhS7v95GzwvRujuOJir4lKjlhCjBDJDBamZQw',
 'accountId': 'UgsvMJCt-77CllcM2LTEmKn8Znx8J8TuaR0rIxOSy2hdn16OarV49TWH',
 'puuid': 'vzO2ozpiICei-jK6WD-59daFWSvv2dH3E9VJ_SrFPLwuRZ7LFdza9Em9Z8gcXgZEi8sYjf6yRydTNQ',
 'name': 'Skeanz',
 'profileIconId': 7,
 'revisionDate': 1667041911000,
 'summonerLevel': 304}

In [6]:
def requestMostRecentGamesId(puuid,key, nb_of_games=None, nb_of_days_ago=None):
    """
    Si nb_of_games    != None : Récupère les 'nb_of_games' dernières parties jouées par le joueur.
    Si nb_of_days_ago != None : Récupère toutes les parties jouées par le joueur dans les 'nb_of_days_ago' jours.
    Si les deux sont != None ou les deux sont == None, renvoie une erreur.
    Renvoie une liste contenant les id de ces parties.
    """
    if (nb_of_games==None and nb_of_days_ago==None) or (nb_of_games != None and nb_of_days_ago != None):
        raise Exception("Erreur d'inputs, les paramètres nb_of_games et nb_of_days_ago ne peuvent pas tous les deux avoir la même valeur")
    elif nb_of_games != None:
        res_games = []
        start_index = 0 # très important, sinon on récupère en boucle les 100 mêmes games
        while nb_of_games >100:    #la valeur max du "count" pour avoir une liste de match est de 100
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count=100&api_key={key}")
            if r == []: # on a atteint le max de games du joueur
                break
            start_index+=100
            res_games += r
            nb_of_games-=100
            
        r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count={nb_of_games}&api_key={key}")
        res_games += r

    elif nb_of_days_ago != None :
        res_games = []
        start_index = 0
        startTime = int(time.time()-24*60*60*nb_of_days_ago) # l'horaire de départ est "aujourd'hui"- nb_of_days_ago si on veut les games dans les nb_of_days_ago derniers jours
        while True: 
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&start={start_index}&count=100&api_key={key}")
            if r == [] : #quand il n'y a plus de games à récup, on sort de la boucle
                break
            start_index+=100
            res_games += r
    return res_games


requestMostRecentGamesId("vzO2ozpiICei-jK6WD-59daFWSvv2dH3E9VJ_SrFPLwuRZ7LFdza9Em9Z8gcXgZEi8sYjf6yRydTNQ",KEY,nb_of_games=10)

['EUW1_6132241704',
 'EUW1_6132256616',
 'EUW1_6132244043',
 'EUW1_6131848101',
 'EUW1_6131611354',
 'EUW1_6131372960',
 'EUW1_6131317851',
 'EUW1_6131176735',
 'EUW1_6131060792',
 'EUW1_6131105739']

In [7]:
def requestMostRecentGamesIdbis(puuid,key, nb_of_games,type_queue='ranked'):
    res_games = []
    r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={nb_of_games}&api_key={key}&type={type_queue}")
    res_games += r
    return res_games
requestMostRecentGamesIdbis("vzO2ozpiICei-jK6WD-59daFWSvv2dH3E9VJ_SrFPLwuRZ7LFdza9Em9Z8gcXgZEi8sYjf6yRydTNQ",KEY,nb_of_games=10)

['EUW1_6132241704',
 'EUW1_6132256616',
 'EUW1_6132244043',
 'EUW1_6131848101',
 'EUW1_6131611354',
 'EUW1_6131372960',
 'EUW1_6131317851',
 'EUW1_6131176735',
 'EUW1_6131060792',
 'EUW1_6131105739']

In [8]:
def requestPlayersOfARank(queue,tier,division,number_of_players,key):
    """ 
    Pour récupérer des joueurs de même ELO

    Args
    ----
    queue : String
        Quelle queue ? Parmis RANKED_SOLO_5x5,RANKED_FLEX_SR,RANKED_FLEX_TT
    tier : String
        Nom de la ligue en anglais (ex : GOLD, PLATINIUM...)
    division : String
        numéro de la division : I,II,III,IV
    number_of_players : Int
        Nombre de joueurs voulus
    
    Returns
    -------
    Dict
        Dictionnaire rassemblant les informations des différents joueurs
        Attention : le nombre de joueurs sera arrondi aux 205 supérieurs (par exemple pour 320, 410 seront donnés)
    """

    tier=tier.upper()
    if number_of_players > 205 : # les pages renvoyées par chaque requête contiennent 205 joueurs
        liste_joueurs = []
        for i in range(1,(number_of_players//205)+2):
            print(i)
            r = badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}I?page={i}&api_key={key}")
            if r == []:
                return liste_joueurs # on a atteint le nombre de joueur total de la division
            liste_joueurs += r
    else :
        liste_joueurs = badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}I?page=1&api_key={key}")
    return liste_joueurs

requestPlayersOfARank("RANKED_SOLO_5x5","DIAMOND","I",10,KEY)

[{'leagueId': '146635dc-9d1b-4f36-a941-3a585dcf12d5',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'II',
  'summonerId': 'Kq6hTyp-t79WpJ4jmf5yR92LZhCO4k6uH7jIbTU4QKrFWnuP',
  'summonerName': 'Veszo',
  'leaguePoints': 43,
  'wins': 293,
  'losses': 270,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'f48de5c1-92ee-451d-8809-2af56753ddeb',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'II',
  'summonerId': 'SQuTpcCXgJn9NHuuh4vxT35uN4JMs_xyVJodHTTc5a90hgg5',
  'summonerName': 'Blood12345678910',
  'leaguePoints': 19,
  'wins': 146,
  'losses': 168,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'fe5aa139-4a22-44a0-bde3-1a861452293b',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'II',
  'summonerId': 'MUPi10zV-3mVUZ0NGqVmTMbW1cvs3QmDd10o51lTTCOycGJh',
  'summonerName': 'Eason Gaming Sam',
  'leaguePoints': 75,
  'wins': 77,


In [9]:
def requestRankedInfo(summoner_id,key):
    """
    Renvoie une liste de dictionnaire contenant des infos sur les résultats du joueur en partie classées, à savoir :
    SOLOQ: {
        tier: string - Iron -> Challenger
        rank: string - IV -> I
        lp: int - league points
        wins: int
        losses: int
    }
    FLEXQ: {pareil}
    """
    data = badRequestsHandler(f"https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}?api_key={key}")
    if len(data)==2 and data[0]['queueType']!='RANKED_SOLO_5x5':
        return [data[1],data[0]]
    return data
    
print(requestRankedInfo('MsJ0SiJEwktncbZVgBlW1uhS7v95GzwvRujuOJir4lKjlhCjBDJDBamZQw',KEY))

[{'leagueId': '04a0f903-37c7-316f-af90-5f8eed42011c', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'CHALLENGER', 'rank': 'I', 'summonerId': 'MsJ0SiJEwktncbZVgBlW1uhS7v95GzwvRujuOJir4lKjlhCjBDJDBamZQw', 'summonerName': 'Skeanz', 'leaguePoints': 1205, 'wins': 901, 'losses': 823, 'veteran': True, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, {'leagueId': '76146db3-5c54-40ba-a6b8-ab4f79f5034e', 'queueType': 'RANKED_FLEX_SR', 'tier': 'PLATINUM', 'rank': 'I', 'summonerId': 'MsJ0SiJEwktncbZVgBlW1uhS7v95GzwvRujuOJir4lKjlhCjBDJDBamZQw', 'summonerName': 'Skeanz', 'leaguePoints': 75, 'wins': 24, 'losses': 24, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


In [10]:
def requestInfoGames(game_id,key):
    """
    Renvoie les informations d'une game
    METADATA: 'matchId'
              'participants' : liste des puuids
    INFO :    'gameDuration'
              'gameEndTimestamp'
              'gameId'
              'gameMode'
              'gameName'
              'gameStartTimestamp'
              'gameType'
              'gameVersion'
              'mapId':
              'participants': liste des informations de la game des 10 participants
              'platformId'
              'queueId'
              'teams': informations globales sur les équipes
    """
    return badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/{game_id}?api_key={key}")

requestInfoGames("EUW1_6131726137",KEY)

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6131726137',
  'participants': ['BYE9lENlwxW6JwTx3xFnvEFPEEc2xXRoZfLZZ8wKCkyx_p6jvTxAeDazRoQf51wh-ELQ4h6ihkhi8Q',
   '9Q4-0M3SkPul8JV2Vx0VMCnNn8tSUVsS3agNkgJ5KjaU9jQ1whIZHNnrIUF-nhwGJgCHI_uFm1mrTg',
   'cqVYkODWdgAYikbR2dl17PTYW80l3JcTqpeqSJjzV8-gQQIM1qSCcdpN-KDJ_SvozuJQKiF0LHIhtA',
   'LviUGGBMgjeu96RiPtQkYYF3xEZMvKDrTdnKgNQ3AMOvyBK8nZI6rxDsT2I2E8DAkNVQu4w_XYX87Q',
   'SXdU5N2RTOF9TsSPE-8FZN_C228moBeiQctdiEFpdwHucle5kxtlpRFRCWDSTnSQ3HsJd4X2z-wHNA',
   'X6BvLX1Ekq-5sUAp-v5SFahyVhW_1HwKKmEmsPFAle8n3kT5rjC-_6HLsmuijs3dpVVIFkaylid8sw',
   '-ABr90__yyR5nb4OLWgs90s8ifBQGWGoe8-uxW8hnNFbBLmI0rzlgBmrwrz7LG3ZeEtQ7IqNMPX0mw',
   'CpYXRvu0dI74hoz_0L2NwojcuII_7DALsa5tEKI9_u0j8_2qBlzmSMbi2utu-5o0QooXW_MrY_mU8Q',
   'Kw1Gd0LV8tqvJOAs2JcHYP2I4_rQuO7zNYpQ-vPATIy4exBxMIKVhhugOxLyaXk4FjMCamVGdnKM1Q',
   'ZKC6uoBYK3dzqU4BD8oZopTmvZTh5nL1PWLvYexsjG1NJQvgfhTply4qebhd4pZTT0g_cTrdRXijWQ']},
 'info': {'gameCreation': 1667249095258,
  'gameDuration': 1761,
 

In [11]:
def getStatsOnLastGames(puuid,n,champion,key):
    """
    Calucle des informations sur les n dernières parties d'un joueur, par rapport à un champion joué sur la partie courante
    KDAG: KDA global moyen sur n games
    KDA: KDA moyen sur le champion, parmis n games (peut être [0,0,0] si le champions n'a pas été joué)
    WR: Winrate sur le champion
    NB: Nombre de games sur le champion
    MOST: Poste le plus joué (pour détrminer si le joueur est fill)
    """
    games = requestMostRecentGamesIdbis(puuid,key,nb_of_games=n)
    KDAG, KDA, WR, NB, MOST, = [0,0,0],[0,0,0],0,0,[]
    for game in games:
        data = requestInfoGames(partie,key)
        participants = data["info"]["participants"] #info des joueurs
        for i in range(10):
            if participants[i]['puuid']==puuid:
                KDAG[0] += participants[i]['kills']
                KDAG[0] += participants[i]['deaths']
                KDAG[0] += participants[i]['assists']
                MOST.append(participants[i]['teamPosition'])
                if participants[i]['championName']==champion:
                    NB += 1
                    WR += participants[i]['win']
                    KDA[0] += participants[i]['kills']
                    KDA[0] += participants[i]['deaths']
                    KDA[0] += participants[i]['assists']
    return [a/n for a in KDAG], [a/n for a in KDA], WR/n, NB, max(set(MOST), key = MOST.count)

## Algorithme général pour la création du Dataset

In [12]:
def infos_games(queue,tier,division,number_of_players,number_of_games,key):
    id_games = []
    liste_joueurs = requestPlayersOfARank("RANKED_SOLO_5x5","DIAMOND","I",number_of_players,key) # chaque élément de la liste contient le nom des joueurs
    liste_joueurs_finale = liste_joueurs[:number_of_players]
    i=0
    while len(id_games) <number_of_games:
        info_joueur = requestSummonerInfo(liste_joueurs_finale[i]['summonerName'],key)
        parties_joueur = requestMostRecentGamesId(info_joueur['puuid'],key, nb_of_games=1000) #recup de toute les parties du joueur (on considère qu'il en a fait - de 100 000)
        id_games+=parties_joueur
        i+=1

    # Une fois les games récup, on supprime les doublons car ils sont inutiles et utilise des requêtes
    id_games_set = set(id_games)
    id_games_uniques = (list(id_games_set))
    if len(id_games_uniques) > number_of_games : # on fait attention de ne pas récup trop de games
        id_games_finale = id_games_uniques[:number_of_games]
    else : 
        id_games_finale = id_games_uniques

    dataset_games = []
    for game in id_games_finale:
        infos_game = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/{game}?api_key={key}")
        dataset_games +=[infos_game]
    
    return(dataset_games)

print(len(infos_games("RANKED_SOLO_5x5","DIAMOND","I",10,200,KEY)))

200


## Calculs de données

In [15]:
df = pd.read_pickle("c:/Users/ptrin/Documents/EI3/LOL/LoL_ML/Création du Dataset/test.pkl")

In [16]:
df

,Y,CHAMP_TOP_0,LVL_TOP_0,TOTAL_TOP_0,GWR_TOP_0,VET_TOP_0,RANK_TOP_0,HOT_TOP_0,KDAG_TOP_0,KDA_TOP_0,...,TOTAL_UTILITY_1,GWR_UTILITY_1,VET_UTILITY_1,RANK_UTILITY_1,HOT_UTILITY_1,KDAG_UTILITY_1,KDA_UTILITY_1,WR_UTILITY_1,NB_UTILITY_1,FILL_UTILITY_1
EUW1_6133084123,False,Tryndamere,39,126,0.555556,False,"[PLATINUM, I, 22]",False,"[12.0, 0.0, 0.0]","[12.0, 0.0, 0.0]",...,1081,0.516189,True,"[DIAMOND, IV, 17]",False,"[20.0, 0.0, 0.0]","[20.0, 0.0, 0.0]",1.0,10,False
EUW1_6125755657,False,Darius,47,108,0.611111,False,"[PLATINUM, I, 100]",False,"[16.0, 0.0, 0.0]","[16.0, 0.0, 0.0]",...,128,0.585938,False,"[PLATINUM, II, 23]",False,"[32.0, 0.0, 0.0]","[32.0, 0.0, 0.0]",1.0,10,False
EUW1_6114981740,True,Yasuo,638,1087,0.502300,True,"[DIAMOND, IV, 29]",False,"[27.0, 0.0, 0.0]","[27.0, 0.0, 0.0]",...,1556,0.504499,False,"[DIAMOND, III, 0]",False,"[33.0, 0.0, 0.0]","[33.0, 0.0, 0.0]",0.0,10,False
EUW1_6076681968,False,KogMaw,40,126,0.595238,False,"[PLATINUM, I, 18]",False,"[11.0, 0.0, 0.0]","[11.0, 0.0, 0.0]",...,586,0.503413,True,"[DIAMOND, II, 9]",False,"[18.0, 0.0, 0.0]","[18.0, 0.0, 0.0]",1.0,10,False
EUW1_6127330820,True,Camille,548,1000,0.508000,False,"[DIAMOND, IV, 40]",False,"[19.0, 0.0, 0.0]","[19.0, 0.0, 0.0]",...,36,0.444444,False,"[GOLD, IV, 30]",False,"[15.0, 0.0, 0.0]","[15.0, 0.0, 0.0]",0.0,10,False
EUW1_6125401150,False,Aatrox,117,233,0.502146,False,"[PLATINUM, II, 45]",False,"[20.0, 0.0, 0.0]","[20.0, 0.0, 0.0]",...,275,0.505455,True,"[DIAMOND, IV, 52]",False,"[24.0, 0.0, 0.0]","[24.0, 0.0, 0.0]",1.0,10,False
EUW1_6132821154,True,Garen,523,460,0.530435,False,"[DIAMOND, III, 50]",False,"[20.0, 0.0, 0.0]","[20.0, 0.0, 0.0]",...,1284,0.514019,True,"[DIAMOND, II, 13]",False,"[26.0, 0.0, 0.0]","[26.0, 0.0, 0.0]",0.0,10,False
EUW1_6131781433,True,Riven,311,375,0.546667,False,"[DIAMOND, II, 3]",False,"[20.0, 0.0, 0.0]","[20.0, 0.0, 0.0]",...,501,0.514970,False,"[MASTER, I, 0]",False,"[24.0, 0.0, 0.0]","[24.0, 0.0, 0.0]",0.0,10,False
EUW1_6115803065,False,MonkeyKing,264,724,0.500000,True,"[DIAMOND, II, 74]",False,"[26.0, 0.0, 0.0]","[26.0, 0.0, 0.0]",...,410,0.543902,False,"[DIAMOND, II, 0]",False,"[36.0, 0.0, 0.0]","[36.0, 0.0, 0.0]",1.0,10,False
EUW1_6130229814,False,Darius,579,704,0.518466,False,"[DIAMOND, III, 4]",False,"[13.0, 0.0, 0.0]","[13.0, 0.0, 0.0]",...,121,0.545455,False,"[DIAMOND, IV, 23]",False,"[37.0, 0.0, 0.0]","[37.0, 0.0, 0.0]",1.0,10,False


In [17]:
def getWinrate(summoner_ID, nb_of_games=0):
    """
    Calcule le winrate (float) et le KDA moyen (float,float,float) sur les 'nb_of_games' dernières parties.
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme:
    {
        wins: int
        losses: int
        winrate: float
        meanKDA: (float,float,float)
    }
    """
    if nb_of_games == 0:
        nb_of_games = 1000
    games = requestMostRecentGamesId(summoner_ID, nb_of_games)
    wins = 0
    losses = 0
    kills = 0
    deaths = 0
    assists = 0
    for game in games:
        if game['win'] == True:
            wins += 1
        else:
            losses += 1
        kills += game['kills']
        deaths += game['deaths']
        assists += game['assists']
    winrate = wins / (wins + losses)
    meanKDA = (kills/len(games), deaths/len(games), assists/len(games))
    return {
        'wins': wins,
        'losses': losses,
        'winrate': winrate,
        'meanKDA': meanKDA
    }

In [18]:
def getWinrateOnChampions(summonerID, nb_of_games=0):
    """
    Calcule le winrate et le KDA moyen sur chaque champion sur les 'nb_of_games' dernières parties
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme :
    {
        Aatrox: {
            wins: int
            losses: int
            winrate: float
            meanKDA: (float,float,float)
        }
        ...
    }
    """
    if nb_of_games == 0:
        nb_of_games = 1000
    games = requestMostRecentGamesId(summonerID, nb_of_games)
    champions = {}
    for game in games:
        if game['champion'] not in champions:
            champions[game['champion']] = {
                'wins': 0,
                'losses': 0,
                'kills': 0,
                'deaths': 0,
                'assists': 0
            }
        if game['win'] == True:
            champions[game['champion']]['wins'] += 1
        else:
            champions[game['champion']]['losses'] += 1
        champions[game['champion']]['kills'] += game['kills']
        champions[game['champion']]['deaths'] += game['deaths']
        champions[game['champion']]['assists'] += game['assists']
    for champion in champions:
        champions[champion]['winrate'] = champions[champion]['wins'] / (champions[champion]['wins'] + champions[champion]['losses'])
        champions[champion]['meanKDA'] = (champions[champion]['kills']/len(games), champions[champion]['deaths']/len(games), champions[champion]['assists']/len(games))
    return champions